In [1]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium as folium
%matplotlib inline

In [2]:
## Load the data into a DataFrame
stores=pd.read_csv('/Users/austinlasseter/DSI-EC-2/projects/datasets/stores_modeling.csv')
stores=stores.drop(['Unnamed: 0'], axis=1)
stores.columns

Index(['store_id', 'fips', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores', 'density', 'unemp_bins', 'income_bins', 'pop_bins',
       'nstores_bins', 'density_bins', 'suburbs', 'town', 'rural'],
      dtype='object')

In [3]:
# Collapse on FIPS code to get the county-level dataset
counties=stores.drop(['store_id', 'metro', 'unemployment', 'income', 'population',
       'ave_bottle_price', 'ave_number_bottles', 'annual_profit_per_store',
       'number_of_stores'], axis=1).groupby('fips', as_index=False).mean()
counties.head()

,fips,density,suburbs,town,rural
0,19001,960.250000,0,0,1
1,19003,4029.000000,0,0,1
2,19005,2388.333333,0,1,0
3,19007,2147.833333,0,1,0
4,19009,2039.666667,0,0,1


In [4]:
# Need to convert fips to a string
print(counties.fips.dtype)
counties['fips']=counties['fips'].astype(str)
print(counties.fips.dtype)

int64
object


In [11]:
counties[counties.fips=='19179'].density.mean()

2544.642857142856

## Map of store density by county

In [6]:
# Read in the json map of all US counties
county_data = '/Users/austinlasseter/DSI-EC-2/projects/folium/tests/us_county_data.csv'
county_geo = '/Users/austinlasseter/DSI-EC-2/projects/folium/tests/us-counties.json'

In [7]:
# Apply a function to map FIPS codes to JSON
def set_id(fips):
    '''Modify FIPS code to match GeoJSON property'''
    if fips == '0':
        return None
    elif len(fips) <= 4:
        return ''.join(['0500000US0', fips])
    else:
        return ''.join(['0500000US', fips])
# Source: https://gist.github.com/wrobstory/5609934

In [8]:
#Apply set_id to our data
counties['GEO_ID'] = counties['fips'].apply(set_id)
counties = counties.dropna()

In [9]:
# Center the map on Iowa
map = folium.Map(location=[42, -93], zoom_start=7)

In [10]:
# Make the map
map.choropleth(geo_data=county_geo, data=counties,
             columns=['fips', 'density'],
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Population per Liquor Store')
map